In [1]:
import lightgbm as lgb
import pandas as pd
from sklearn.metrics import confusion_matrix,accuracy_score, roc_curve, auc
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
import string
import re

from time import time

In [2]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [85]:
id='1Pk5MK9Hs_kMUT9NotGnOKE0NPra-39YU'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('train.csv')

train = pd.read_csv('train.csv')

id='1GsTM9oLtIV8-Da_fDOFWsQYMpgQ8GOYJ'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('test.csv')

test = pd.read_csv('test.csv')

In [84]:
id='1slJiG-M_oawvkzaqWzZkSmVxxZor6mKQ'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('train_features.csv')

id='1TmL2AY_yymiV8zXVvmbqXsIZoexR9sLs'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('test_features.csv')

train_features = pd.read_csv('train_features.csv')
test_features = pd.read_csv('test_features.csv')

In [82]:
id= '1Ffgm1RoYdjqMuQgTap3o5bOSaNjIdO7l'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('matriz_test_doc2vec_solo_text.csv')

id= '1Q7m92Pbm08m0t9X8Q975IUkPWEoZQ--R'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('matriz_train_doc2vec_solo_text.csv')

In [83]:
matriz_test_doc2vec_solo_text = pd.read_csv('matriz_test_doc2vec_solo_text.csv')
matriz_train_doc2vec_solo_text = pd.read_csv('matriz_train_doc2vec_solo_text.csv')

y_train = train['target']

Parametros LightGBM

In [ ]:
params = {
    'application': 'binary',
    'objective': 'binary',
    'metric': 'auc',
    'is_unbalance': 'false',
    'boosting': 'gbdt',
    'num_leaves': 31,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.5,
    'bagging_freq': 20,
    'learning_rate': 0.05,
    'verbose': 0
}

#DOC2VEC

###Solo con texto

In [ ]:
lgb_train = lgb.Dataset(matriz_train_doc2vec_solo_text , y_train)

In [ ]:
gbm = lgb.train(params, lgb_train, num_boost_round=50)

In [ ]:
predictions = gbm.predict(matriz_test_doc2vec_solo_text)

array([0.49451934, 0.48024837, 0.55287008, ..., 0.40741218, 0.42201578,
       0.42770756])

In [ ]:
predictions_classes = []
for i in predictions:
  predictions_classes.append(round(i))
 
predictions_classes = np.array(predictions_classes)

In [ ]:
predictions_classes

array([0., 0., 1., ..., 0., 0., 0.])

##Con todos los features

In [ ]:
id= '1OrAkBep-bmOz-P-QR3P5csDejgiJG9bL'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('doc2vec_features_numericos_test.csv')

id= '17bNeSA9q59vTO6OW2lBTJPyw29QGQSxf'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('doc2vec_features_numericos_train.csv')

In [ ]:
doc2vec_features_numericos_test = pd.read_csv('doc2vec_features_numericos_test.csv')
doc2vec_features_numericos_train = pd.read_csv('doc2vec_features_numericos_train.csv')

In [ ]:
lgb_train = lgb.Dataset(doc2vec_features_numericos_train , y_train)

In [ ]:
gbm = lgb.train(params, lgb_train, num_boost_round=50)

In [ ]:
predictions = gbm.predict(doc2vec_features_numericos_test)

In [ ]:
predictions_classes = []
for i in predictions:
  predictions_classes.append(round(i))
 
predictions_classes = np.array(predictions_classes)

In [ ]:
predictions_classes

array([0., 0., 0., ..., 1., 1., 1.])

In [ ]:
#DIO 0.62!
submission_lightGBM = pd.DataFrame(data={'id':test['id'], 'target': predictions_classes})
submission_lightGBM['target'] = submission_lightGBM['target'].astype(int)
submission_lightGBM.to_csv('lightGBM.csv', index=False)

#Doc2Vec solo texto + keyword Categorical

FEATURES: TEXT Y KEYWORD

In [59]:
x_train = train.drop(columns=['id', 'target', 'location', 'text'])
x_train['keyword'].fillna(' ', inplace=True)
x_train.dtypes

keyword    object
dtype: object

In [60]:
x_train['keyword'] = x_train['keyword'].astype('category')
x_train['keyword'] = pd.Categorical(x_train['keyword'].cat.codes+1)

In [61]:
features_text_keyword = pd.concat([x_train, matriz_train_doc2vec_solo_text], axis=1)

In [63]:
train_data = lgb.Dataset(features_text_keyword , label=y_train, categorical_feature=['keyword'])

In [64]:
gbm = lgb.train(params, train_data, num_boost_round=50)

/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


In [65]:
x_test = test.drop(columns=['id', 'location', 'text'])
x_test['keyword'].fillna(' ', inplace=True)
x_test.dtypes

keyword    object
dtype: object

In [66]:
x_test['keyword'] = x_test['keyword'].astype('category')
x_test['keyword'] = pd.Categorical(x_test['keyword'].cat.codes+1)

In [67]:
test_text_keyword = pd.concat([x_test, matriz_test_doc2vec_solo_text], axis=1)

In [68]:
predictions = gbm.predict(test_text_keyword)

In [69]:
predictions_classes = []
for i in predictions:
  predictions_classes.append(round(i))
 
predictions_classes = np.array(predictions_classes)

In [70]:
#DIO 0.70977!
submission_lightGBM_2 = pd.DataFrame(data={'id':test['id'], 'target': predictions_classes})
submission_lightGBM_2['target'] = submission_lightGBM_2['target'].astype(int)
submission_lightGBM_2.to_csv('lightGBM_2.csv', index=False)

FEATURES: TEXT, KEYWORD Y NUMERICOS

In [86]:
train_features = train_features.drop(columns=['id', 'location', 'text', 'keyword', 'target', 'longitud tweets', 'cant_palabras', 'palabras_unicas', 'cant_url', 'cant_hashtag', 'cant_mencion',	'cant_signos_pregunta',	'cant_signos_exclamacion', 'subjetividad', 'cant_oraciones', 'cant_minusculas', 'cant_consonant'])

In [87]:
features_text_keyword_numeric = pd.concat([x_train, matriz_train_doc2vec_solo_text, train_features], axis=1)

In [88]:
train_data_2 = lgb.Dataset(features_text_keyword_numeric, label=y_train, categorical_feature=['keyword', 'cant_stop_words', 'prom_long_palabra', 'cant_puntuacion', 'cant_apariciones_keyword', 'cant_numeros', 'cant_mayusculas', 'cant_vocales'])

In [90]:
gbm = lgb.train(params, train_data_2, num_boost_round=50)

/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


In [92]:
test_features = test_features.drop(columns=['id', 'location', 'text', 'keyword', 'longitud tweets', 'cant_palabras', 'palabras_unicas', 'cant_url', 'cant_hashtag', 'cant_mencion',	'cant_signos_pregunta',	'cant_signos_exclamacion', 'subjetividad', 'cant_oraciones', 'cant_minusculas', 'cant_consonant'])

In [97]:
test_text_keyword_2 = pd.concat([x_test, matriz_test_doc2vec_solo_text, test_features], axis=1)

In [98]:
predictions = gbm.predict(test_text_keyword_2)

In [99]:
predictions_classes = []
for i in predictions:
  predictions_classes.append(round(i))
 
predictions_classes = np.array(predictions_classes)

In [100]:
#DIO 0.70977!
submission_lightGBM_3 = pd.DataFrame(data={'id':test['id'], 'target': predictions_classes})
submission_lightGBM_3['target'] = submission_lightGBM_3['target'].astype(int)
submission_lightGBM_3.to_csv('lightGBM_3.csv', index=False)

In [102]:
accuracy_score(submission_lightGBM_2['target'], submission_lightGBM_3['target'])

0.9010113392583512